In [26]:
#pip install plotly==4.1.0

import plotly.express as px
import plotly.graph_objs as go

import pandas as pd
import numpy as np
from ipywidgets import interactive, HBox, VBox
from ipywidgets import interact

import math
from math import sqrt


In [2]:
from plotly.subplots import make_subplots
from plotly import tools

clasificar_j = pd.read_csv('ENORTE.csv', delimiter = ',') ## Import the dataset desde sklearn.datasets

#clasificar_j.head()

In [162]:
from ipywidgets import widgets
from IPython.display import clear_output

t = np.linspace(-1,1.2,2000)
x = clasificar_j['s3']
y = clasificar_j['s1']

def SetColor(x):
    if(x == 0):
        return 'rgb(0, 200, 200)'
    elif(x == 1):
        return "red"
    
def NameUG(x):
    if(x == 0):
        return "Used Data"
    elif(x == 1):
        return "Discarded"
    
def UCSUG(x):
    if(x == 0):
        return "Used UCS"
    elif(x == 1):
        return "Discarded UCS"

s3j = [c - 10 for c in range(41)]
    
def Hoek(x):
    if(x=="TODOLIX"):
        b=10
        a=24.4
        s1j = [(n + a * (sqrt(b * (n / a) + 1))) for n in s3j if (b * (n / a) + 1)>=0]
        s3p = [(n) for n in s3j if (b * (n / a) + 1)>=0]
    if(x=="PRIM"):
        b=15
        a=35.9
        s1j = [(n + a * (sqrt(b * (n / a) + 1))) for n in s3j if (b * (n / a) + 1)>=0]
        s3p = [(n) for n in s3j if (b * (n / a) + 1)>=0]
    if(x=="SEC"):
        b=9
        a=33.5
        s1j = [(n + a * (sqrt(b * (n / a) + 1))) for n in s3j if (b * (n / a) + 1)>=0]
        s3p = [(n) for n in s3j if (b * (n / a) + 1)>=0]
    if(x=="PRIMSEC"):
        b=9
        a=33.5
        s1j = [(n + a * (sqrt(b * (n / a) + 1))) for n in s3j if (b * (n / a) + 1)>=0]
        s3p = [(n) for n in s3j if (b * (n / a) + 1)>=0]
    return s3p, s1j, a, b
        
        


## definicion del grafico mediante plotly, la variable a cambiar corresponde a "hue"

def plotpro(Select):

    fig = make_subplots(rows=2, cols=3, specs=[[{"type": "bar"},None, {"type": "scatter"}],
                                               [None, None, {"type": "scatter"}]], column_widths=[0.2, 0.1, 0.7], row_heights=[1, 1])

  
    for UG, stress in clasificar_j.groupby(Select):
        fig.add_trace(go.Scatter(x=stress.s3, y=stress.s1, name=NameUG(UG), mode='markers',
                                 marker=dict(size=10, opacity=0.7, line=dict(width=1), color=SetColor(UG))), row=1, col=3)
        fig.update_xaxes(title_text="Sigma 3 (Mpa)",titlefont=dict(family='Rockwell',size=15,color='rgb(17, 60, 160)'), range=[-10, 30], row=1, col=3)
        fig.update_yaxes(title_text="Sigma 1 (Mpa)",titlefont=dict(family='Rockwell',size=15,color='rgb(17, 60, 160)'), range=[0, 350], row=1, col=3)

        fig.add_trace(go.Histogram(y=stress.s1, name= UCSUG(UG), xaxis='x2', yaxis='y2', opacity=0.6, marker_color=SetColor(UG)), row=1, col=1)
        
        # Agregamos los puntos en las coordenadas del rajo
        fig.add_trace(
            go.Scatter(x=(stress.x-17000)/1667, y=(stress.y-112000)/909, mode='markers', showlegend=False,
                       marker=dict(color=SetColor(UG), size=10, opacity=0.3, line=dict(width=1, color='DarkSlateGrey')), xaxis='x3', yaxis='y3'), row=2, col=3)
        
        # Constants, obtenidos de las propiedades de la imagen..
        img_width = 11342
        img_height = 12431
        scale_factor = 0.0002
        
        
    # agregamos ajuste por Hoek
    fig.append_trace(go.Scatter(x=Hoek(Select)[0], y=Hoek(Select)[1], name='Hoek Fit: Sci=%4.1f' % Hoek(Select)[2]+ ' , mi=%4.1f' % Hoek(Select)[3], mode='lines', line = dict(color='firebrick', width=4, dash='dot')), 1, 3)
        
    fig.update_xaxes(title_text="Histogram S1 (Mpa)",titlefont=dict(family='Rockwell',size=15,color='rgb(17, 60, 160)'), range=[30, 0], mirror=True, row=1, col=1)
    fig.update_yaxes(range=[0, 350], showticklabels=False, row=1, col=1)
    fig.update_xaxes(showline=True, linewidth=1, linecolor='black')
    fig.update_yaxes(showline=True, linewidth=1, linecolor='black')
    fig.layout.update(title_text="Curve Fitting for Generalized Hoek-Brown",width = 800, height=800)
    
    # Add images
    fig.layout.update(
        images=[
            go.layout.Image(
                source="Planta_EscondidaNorte_EOY23.png",
                xref="x3",
                yref="y3",
                x=0,
                y=3,
                sizex=img_width * scale_factor,
                sizey=img_height * scale_factor,
                sizing="stretch",
                opacity=1.0,
                layer="below")])
    fig.layout.update(
        yaxis3 = dict(scaleanchor = "y3", scaleratio = 1, range=[0.5, 3], domain=[0, 0.45], showgrid=False,showticklabels=False, mirror=True),plot_bgcolor='rgba(228, 222, 249, 0.65)',
        xaxis3 = dict(scaleanchor = "x3", scaleratio = 1, range=[0, 2.25], domain=[0.45, 0.9], showgrid=False, showticklabels=False, mirror=True))


    fig.show()
    
################# codigo para utilizar: from ipywidgets import widgets. ################

#origin = widgets.Dropdown(
#    options=["TODOLIX", "PRIM", "PRIMSEC", "SEC"],
#    value='TODOLIX',
#    description='Select Chart:',
#)

#display(origin)
#output = widgets.Output()
#display(output)

#def on_change(change):
#    with output:
#        if change['type'] == 'change' and change['name'] == 'value':
#            clear_output()
#            plotpro(change['new'])

#origin.observe(on_change)        

######################## fin from ipywidgets import widgets ##########################


In [163]:
%%javascript
IPython.OutputArea.auto_scroll_threshold = 9999;

<IPython.core.display.Javascript object>

In [164]:
interact(plotpro,description='Test:', Select={'Lixiviation':"TODOLIX", 'Primary Mineralization':"PRIM", 'Secondary Mineralization':"SEC", 'Primary and Secondary Mineralization':"PRIMSEC"})

interactive(children=(Dropdown(description='Select', options={'Lixiviation': 'TODOLIX', 'Primary Mineralizatio…

<function __main__.plotpro(Select)>